# Road Test

In [ ]:
import emat
emat.__version__

In [ ]:
import ema_workbench
import os, numpy, pandas, functools
from emat.util.xmle import Show
from emat.viz.scatter import scatter_graph

In [ ]:
logger = emat.util.loggers.log_to_stderr(20, True)

## Defining the Exploratory Scope

In [ ]:
road_test_scope_file = emat.package_file('model','tests','road_test.yaml')

In [ ]:
road_scope = emat.Scope(road_test_scope_file)
road_scope

A short summary of the scope can be reviewed using the `info` method.

In [ ]:
road_scope.info()

Alternatively, more detailed information about each part of the scope can be
accessed in four list attributes:

In [ ]:
road_scope.get_constants()

In [ ]:
road_scope.get_uncertainties()

In [ ]:
road_scope.get_levers()

In [ ]:
road_scope.get_measures()

## Using a Database

The exploratory modeling process will typically generate many different sets of outputs,
for different explored modeling scopes, or for different applications.  It is convenient
to organize these outputs in a database structure, so they are stored consistently and 
readily available for subsequent analysis.

The `SQLiteDB` object will create a database to store results.  When instantiated with
no arguments, the database is initialized in-memory, which will not store anything to
disk (which is convenient for this example, but in practice you will generally want to
store data to disk so that it can persist after this Python session ends).

In [ ]:
emat_db = emat.SQLiteDB()

An EMAT Scope can be stored in the database, to provide needed information about what the 
various inputs and outputs represent.

In [ ]:
road_scope.store_scope(emat_db)

Trying to store another scope with the same name (or the same scope) raises a KeyError.

In [ ]:
try:
    road_scope.store_scope(emat_db)
except KeyError as err:
    print(err)

We can review the names of scopes already stored in the database using the `read_scope_names` method.

In [ ]:
emat_db.read_scope_names()

## Experimental Design

Actually running the model can be done by the user on an *ad hoc* basis (i.e., manually defining every 
combination of inputs that will be evaluated) but the real power of EMAT comes from runnning the model
using algorithm-created experimental designs.

An important experimental design used in exploratory modeling is the Latin Hypercube.  This design selects
a random set of experiments across multiple input dimensions, to ensure "good" coverage of the 
multi-dimensional modeling space.

The `design_latin_hypercube` function creates such a design based on a `Scope`, and optionally
stores the design of experiments in a database.

In [ ]:
from emat.experiment.experimental_design import design_experiments

In [ ]:
design = design_experiments(road_scope, db=emat_db, n_samples_per_factor=10, sampler='lhs')
design.head()

In [ ]:
large_design = design_experiments(road_scope, db=emat_db, n_samples=5000, sampler='lhs', design_name='lhs_large')
large_design.head()

We can review what experimental designs have already been stored in the database using the 
`read_design_names` method of the `Database` object.

In [ ]:
emat_db.read_design_names('EMAT Road Test')

## Core Model in Python

### Model Definition

In the simplest approach for EMAT, a model can be defined as a basic Python function, which accepts all
inputs (exogenous uncertainties, policy levers, and externally defined constants) as named keyword
arguments, and returns a dictionary where the dictionary keys are names of performace measures, and 
the mapped values are the computed values for those performance measures.  The `Road_Capacity_Investment`
function provided in EMAT is an example of such a function.  This made-up example considers the 
investment in capacity expansion for a single roadway link.  The inputs to this function are described
above in the Scope, including uncertain parameters in the volume-delay function,
traffic volumes, value of travel time savings, unit construction costs, and interest rates, and policy levers including the 
amount of capacity expansion and amortization period.

In [ ]:
from emat.model.core_python import PythonCoreModel
from emat.model.core_python import Road_Capacity_Investment

In [ ]:
from emat.model.core_python import PythonCoreModel
m = PythonCoreModel(Road_Capacity_Investment, scope=road_scope, db=emat_db)

### Model Execution

In [ ]:
from ema_workbench import SequentialEvaluator

In [ ]:
with SequentialEvaluator(m) as eval_seq:
    lhs_results = m.run_experiments(design_name='lhs', evaluator=eval_seq)
lhs_results.head()

In [ ]:
# with SequentialEvaluator(m) as eval_seq:
#     lhs_large_results = m.run_experiments(design_name='lhs_large', evaluator=eval_seq)
# lhs_large_results.head()

Once a particular design has been run once, the results can be recovered from the database without re-running the model itself.

In [ ]:
reload_results = m.read_experiments(design_name='lhs')
reload_results.head()

It is also possible to load only the parameters, or only the performance meausures.

In [ ]:
lhs_params = m.read_experiment_parameters(design_name='lhs')
lhs_params.head()

In [ ]:
lhs_outcomes = m.read_experiment_measures(design_name='lhs')
lhs_outcomes.head()

### CART

Classification and Regression Trees (CART) can also be used for scenario discovery. 
They partition the explored space (i.e., the scope) into a number of sections, with each partition
being added in such a way as to maximize the difference between observations on each 
side of the newly added partition divider, subject to some constraints.

In [ ]:
# from ema_workbench.analysis import cart

# cart_alg = cart.CART(
#     m.read_experiment_parameters(design_name='lhs_large'), 
#     m.read_experiment_measures(design_name='lhs_large')['net_benefits']>0,
# )
# cart_alg.build_tree()

In [ ]:
# Show(cart_alg.show_tree(format='svg')) 

In [ ]:
# cart_alg.boxes_to_dataframe(include_stats=True)

# Constraints

In [ ]:
from emat import Constraint

The common use case for constraints in robust optimation is imposing requirements
on solution outcomes. For example, we may want to limit our robust search only
to solutions where the expected present cost of the capacity expansion is less
than some particular value (in our example here, 4000).  

In [ ]:
constraint_1 = Constraint(
    "Maximum build_travel_time", 
    outcome_names="build_travel_time",
    function=Constraint.must_be_less_than(70),
)

Our second constraint is based exclusively on an input: the capacity expansion
must be at least 10.  We could also achieve this kind of constraint by changing
the exploratory scope, but we don't necessarily want to change the scope to 
conduct a single robust optimization analysis with a constraint on a policy lever.

In [ ]:
constraint_2 = Constraint(
    "Minimum Capacity Expansion", 
    parameter_names="expand_capacity",
    function=Constraint.must_be_greater_than(20),
)

It is also possible to impose constraints based on a combination of inputs and outputs.
For example, suppose that the total funds available for pay-as-you-go financing are
only 1500.  We may thus want to restrict the robust search to only solutions that
are almost certainly within the available funds at 99% confidence (a model output) but only 
if the Paygo financing option is used (a model input).  This kind of constraint can
be created by giving both `parameter_names` and `outcomes_names`, and writing a constraint
function that takes two arguments.

In [ ]:
constraint_3 = Constraint(
    "Maximum Paygo Present Cost", 
    parameter_names='debt_type',
    outcome_names='present_cost_expansion',
    function=lambda i,j: max(0, j-4000) if i=='Paygo' else 0,
)

In [ ]:
constraints=[
            constraint_1,
            constraint_2,
            constraint_3,
        ]

In [ ]:
from emat.util.constraints import batch_contraint_check

In [ ]:
batch_contraint_check(constraints, lhs_params, lhs_outcomes, False).head()

In [ ]:
from emat.scope.box import Bounds, Box, Boxes, find_all_boxes_with_parent

In [ ]:
db = emat_db

In [ ]:
try:
    s = Box(scope=road_scope)
except TypeError:
    print("correct error")
else:
    raise RuntimeError

In [ ]:
s = Box(
    name="Speedy", 
    scope=road_scope,
    upper_bounds={'build_travel_time':70},
    relevant=['net_benefits', 'time_savings'],
)

In [ ]:
s2 = Box(
    name="Notable", 
    scope=road_scope, 
    parent="Speedy",
    lower_bounds={'expand_capacity': 20},
    relevant=road_scope.get_lever_names(),
)

In [ ]:
s3 = Box(
    name="No Tax Dollars",
    scope=road_scope, 
    parent="Notable",
    allowed={
        'debt_type': {'Paygo', 'Rev Bond'},
        'interest_rate_lock': {False},
    }
)

In [ ]:
u = Boxes(s,s2,s3, scope=road_scope)

In [ ]:
u.fancy_names()

In [ ]:
u.get_chain("No Tax Dollars")

In [ ]:
print(u.get_chain("No Tax Dollars").chain_repr())

In [ ]:
db.write_boxes(u)

In [ ]:
uu = db.read_boxes()

In [ ]:
uu.fancy_names()

In [ ]:
uu.get_chain("No Tax Dollars")

In [ ]:
db.read_scope('EMAT Road Test')

In [ ]:
from emat.interactive import Explorer

In [ ]:
explore = Explorer('db')
explore

In [ ]:
from emat.interactive import prototype_logging
prototype_logging.handler.out

In [ ]:
prototype_logging.logger.setLevel(10)

In [ ]:
True in {0}